## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Sep. 30th Oct. 7th


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,October-2-2024,Condo/Co-op,1450 Lincoln Ter #1,Miami Beach,FL,33139.0,670000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1450-Lin...,MARMLS,A11634369,N,Y,25.789386,-80.144286


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
70,PAST SALE,2024-10-04,Condo/Co-op,11259 SW 88 St Unit 104-H,Miami,FL,33176.0,298700.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/11259-N-Kendal...,MARMLS,A11593733,N,Y,25.688450,-80.374911
77,PAST SALE,2024-10-04,Condo/Co-op,2203 SE 27th Dr Unit 102-A,Homestead,FL,33035.0,215000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Homestead/2203-SE-27...,MARMLS,A11583740,N,Y,25.450719,-80.437495
50,PAST SALE,2024-10-04,Condo/Co-op,8775 Park Blvd #411,Miami,FL,33172.0,305000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/8775-Park-Blvd...,MARMLS,A11601887,N,Y,25.773541,-80.338252
51,PAST SALE,2024-10-04,Condo/Co-op,500 Bayview Dr #728,Sunny Isles Beach,FL,33160.0,305000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/50...,MARMLS,A11629781,N,Y,25.922376,-80.127190
55,PAST SALE,2024-10-04,Condo/Co-op,15325 SW 106th Ter #609,Miami,FL,33196.0,215000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/15325-SW-106th...,MARMLS,A11612104,N,Y,25.667628,-80.444757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,PAST SALE,2024-09-30,Condo/Co-op,13953 Kendale Lakes Cir Unit 208B,Miami,FL,33183.0,390000.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/13953-Kendale-...,MARMLS,A11587306,N,Y,25.706342,-80.421354
89,PAST SALE,2024-09-30,Condo/Co-op,1425 Brickell Ave Unit 57E,Miami,FL,33131.0,4500000.0,4.0,4.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1425-Brickell-...,MARMLS,A11578904,N,Y,25.759132,-80.191829
90,PAST SALE,2024-09-30,Condo/Co-op,9381 E Bay Harbor Dr Unit 401N,Bay Harbor Islands,FL,33154.0,325000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Bay-Harbor-Islands/9...,MARMLS,A11575000,N,Y,25.884755,-80.128573
64,PAST SALE,2024-09-30,Condo/Co-op,3625 N Country Club Dr #2608,Aventura,FL,33180.0,355000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/3625-N-Country...,MARMLS,A11610502,N,Y,25.966677,-80.129515


In [10]:
df = df.sort_values(by='PRICE',ascending=False)

In [69]:
# df = df.drop(index=64)

In [11]:
df_top_ten = df.head(10)

## Collect Agent Information

In [12]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [13]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [14]:
response_df = pd.DataFrame(response_list)

In [15]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [16]:
df_top_ten = merged_df

## Current Week's Values

In [17]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

120


In [18]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$625,291


In [19]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$501


In [20]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$75,034,917


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [27]:
# Define the date range
# start_date = '2024-09-08'
# end_date = '2024-09-15'

# Filter the DataFrame between two dates (inclusive)
# filtered_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [21]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Miami-Dade County
Input Previous Week Condo Sales Total (number units sold):
120
Input Previous Week Condo Average Sales Price:
656812
Input Previous Week Condo Average PSF:
475
Input Previous Week Condo Sales total (ex: 198_000_000)
93_800_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [22]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [23]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [24]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [25]:
pd.set_option('display.max_columns',None)

In [26]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-10-04,Condo/Co-op,1000 S Pointe Dr #301,Miami Beach,FL,33139.0,6850000.0,3.0,4.5,MURANO AT PORTOFINO CONDO,3365.0,NaN,2002.0,NaN,2036.0,4631.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,MARMLS,A11569150,N,Y,25.768527,-80.137591,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,Denise Santos,"Compass Florida, LLC.",None,None,Sildy Cervera,Cervera Real Estate Inc.,None,None,1,orange


In [27]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [28]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-10-04,Condo/Co-op,1000 S Pointe Dr #301,Miami Beach,FL,33139.0,6850000.0,3.0,4.5,MURANO AT PORTOFINO CONDO,3365.0,NaN,2002.0,NaN,2036.0,4631.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,MARMLS,A11569150,N,Y,25.768527,-80.137591,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,Denise Santos,"Compass Florida, LLC.",None,None,Sildy Cervera,Cervera Real Estate Inc.,None,None,1,orange
1,PAST SALE,2024-09-30,Condo/Co-op,1425 Brickell Ave Unit 57E,Miami,FL,33131.0,4500000.0,4.0,4.0,MILLENNIUM TOWER RESIDENCE,3357.0,NaN,2003.0,NaN,1340.0,4673.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1425-Brickell-...,MARMLS,A11578904,N,Y,25.759132,-80.191829,https://www.redfin.com/FL/Miami/1425-Brickell-...,Darren Weiner,Celebrity Advisors,None,None,Shakira Sanchez,Douglas Elliman,None,None,2,blue
2,PAST SALE,2024-10-03,Condo/Co-op,2127 Brickell Ave #3405,Miami,FL,33129.0,2475000.0,3.0,3.0,BRISTOL TOWER CONDO,2310.0,NaN,1993.0,NaN,1071.0,3902.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2127-Brickell-...,MARMLS,A11613620,N,Y,25.752160,-80.199342,https://www.redfin.com/FL/Miami/2127-Brickell-...,Liz Hogan,"Compass Florida, LLC.",None,None,Mariana Niro,Serhant,None,None,3,blue
3,PAST SALE,2024-10-04,Condo/Co-op,300 Biscayne Blvd Way #702,Miami,FL,33131.0,2400000.0,2.0,3.0,Aston Martin,1474.0,NaN,2024.0,NaN,1628.0,2588.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/300-Biscayne-B...,MARMLS,A11594800,N,Y,25.770777,-80.187521,https://www.redfin.com/FL/Miami/300-Biscayne-B...,Mickael Lancri,Mickael Lancri PA,None,None,NON-MLS MEMBER,MAR NON MLS MEMBER,None,None,4,blue
4,PAST SALE,2024-10-01,Condo/Co-op,801 S Miami Ave Ph 5601,Miami,FL,33130.0,2100000.0,3.0,3.5,801 SMA RESIDENCES CONDO,1996.0,NaN,2018.0,NaN,1052.0,3250.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/801-S-Miami-Av...,MARMLS,A11595899,N,Y,25.765990,-80.193097,https://www.redfin.com/FL/Miami/801-S-Miami-Av...,Reid Heidenry,One Sotheby's International Re,Marko Gojanovic,One Sotheby's International Re,Marko Gojanovic,One Sotheby's International Re,None,None,5,blue
5,PAST SALE,2024-10-01,Condo/Co-op,3301 NE 183rd St #2306,Aventura,FL,33160.0,1745000.0,4.0,4.5,THE PENINSULA II CONDO,3524.0,NaN,2007.0,NaN,495.0,3055.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/3301-NE-183...,MARMLS,A11560683,N,Y,25.946197,-80.137058,https://www.redfin.com/FL/Aventura/3301-NE-183...,Lai Ming Lam,United Realty Group Inc,None,None,Geraldine Dayan,"Compass Florida, LLC.",None,None,6,blue
6,PAST SALE,2024-09-30,Condo/Co-op,18201 Collins Ave #4308,Sunny Isles Beach,FL,33160.0,1550000.0,2.0,2.5,TRUMP ROYALE CONDO,1570.0,NaN,2008.0,NaN,987.0,1587.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11559748,N,Y,25.944933,-80.120001,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Mariana Shulga,"Mariana Shulga, LLC.",None,None,Gustavo Strallnikoff,Dezer Platinum Realty LLC,None,None,7,blue
7,PAST SALE,2024-09-30,Condo/Co-op,3400 SW 27th Ave #605,Coconut Grove,FL,33133.0,1450000.0,2.0,2.5,THE TOWER RESIDENCES,1540.0,NaN,2001.0,NaN,942.0,2729.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/3400-SW-27th-A...,MARMLS,A11506988,N,Y,25.729664,-80.237811,https://www.redfin.com/FL/Miami/3400-SW-27th-A...,Glen Janney,Cervera Real Estate Inc.,None,None,Dirk Saecker,"Four Corners Real Estate, LLC.",None,None,8,blue
8,PAST SALE,2024-10-01,Condo/Co-op,700 NE 26th Ter #4606,Miami,FL,33137.0,1325000.0,2.0,2.0,Missoni Baia,1257.0,NaN,2023.0,NaN,1054.0,1835.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,MARM

In [29]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} - {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [30]:
m.save('index.html')

## Data snagger

In [31]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [32]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-10-04,Condo/Co-op,1000 S Pointe Dr #301,Miami Beach,FL,33139.0,6850000.0,3.0,4.5,MURANO AT PORTOFINO CONDO,3365.0,NaN,2002.0,NaN,2036.0,4631.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,MARMLS,A11569150,N,Y,25.768527,-80.137591,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,Denise Santos,"Compass Florida, LLC.",None,None,Sildy Cervera,Cervera Real Estate Inc.,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [33]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MD_County_condo_sales_week_ending_10072024


# CREATE TEMPLATE 

In [34]:
muni_set = set(df_top_ten['CITY'])

In [35]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [36]:
df_top_ten.reset_index(inplace=True,drop=True)

In [37]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [38]:
top_sale

'MURANO AT PORTOFINO CONDO at 1000 S Pointe Dr #301 in Miami Beach'

In [39]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [40]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [41]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-10-04,Condo/Co-op,1000 S Pointe Dr #301,Miami Beach,FL,33139.0,6850000.0,3.0,4.5,MURANO AT PORTOFINO CONDO,3365.0,NaN,2002.0,NaN,2036.0,4631.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,MARMLS,A11569150,N,Y,25.768527,-80.137591,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,Denise Santos,"Compass Florida, LLC.",None,None,Sildy Cervera,Cervera Real Estate Inc.,None,None,1,orange,MURANO AT PORTOFINO CONDO at 1000 S Pointe Dr ...
1,PAST SALE,2024-09-30,Condo/Co-op,1425 Brickell Ave Unit 57E,Miami,FL,33131.0,4500000.0,4.0,4.0,MILLENNIUM TOWER RESIDENCE,3357.0,NaN,2003.0,NaN,1340.0,4673.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1425-Brickell-...,MARMLS,A11578904,N,Y,25.759132,-80.191829,https://www.redfin.com/FL/Miami/1425-Brickell-...,Darren Weiner,Celebrity Advisors,None,None,Shakira Sanchez,Douglas Elliman,None,None,2,blue,MILLENNIUM TOWER RESIDENCE at 1425 Brickell Av...
2,PAST SALE,2024-10-03,Condo/Co-op,2127 Brickell Ave #3405,Miami,FL,33129.0,2475000.0,3.0,3.0,BRISTOL TOWER CONDO,2310.0,NaN,1993.0,NaN,1071.0,3902.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2127-Brickell-...,MARMLS,A11613620,N,Y,25.752160,-80.199342,https://www.redfin.com/FL/Miami/2127-Brickell-...,Liz Hogan,"Compass Florida, LLC.",None,None,Mariana Niro,Serhant,None,None,3,blue,BRISTOL TOWER CONDO at 2127 Brickell Ave #3405...
3,PAST SALE,2024-10-04,Condo/Co-op,300 Biscayne Blvd Way #702,Miami,FL,33131.0,2400000.0,2.0,3.0,Aston Martin,1474.0,NaN,2024.0,NaN,1628.0,2588.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/300-Biscayne-B...,MARMLS,A11594800,N,Y,25.770777,-80.187521,https://www.redfin.com/FL/Miami/300-Biscayne-B...,Mickael Lancri,Mickael Lancri PA,None,None,NON-MLS MEMBER,MAR NON MLS MEMBER,None,None,4,blue,Aston Martin at 300 Biscayne Blvd Way #702 in ...
4,PAST SALE,2024-10-01,Condo/Co-op,801 S Miami Ave Ph 5601,Miami,FL,33130.0,2100000.0,3.0,3.5,801 SMA RESIDENCES CONDO,1996.0,NaN,2018.0,NaN,1052.0,3250.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/801-S-Miami-Av...,MARMLS,A11595899,N,Y,25.765990,-80.193097,https://www.redfin.com/FL/Miami/801-S-Miami-Av...,Reid Heidenry,One Sotheby's International Re,Marko Gojanovic,One Sotheby's International Re,Marko Gojanovic,One Sotheby's International Re,None,None,5,blue,801 SMA RESIDENCES CONDO at 801 S Miami Ave Ph...
5,PAST SALE,2024-10-01,Condo/Co-op,3301 NE 183rd St #2306,Aventura,FL,33160.0,1745000.0,4.0,4.5,THE PENINSULA II CONDO,3524.0,NaN,2007.0,NaN,495.0,3055.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/3301-NE-183...,MARMLS,A11560683,N,Y,25.946197,-80.137058,https://www.redfin.com/FL/Aventura/3301-NE-183...,Lai Ming Lam,United Realty Group Inc,None,None,Geraldine Dayan,"Compass Florida, LLC.",None,None,6,blue,THE PENINSULA II CONDO at 3301 NE 183rd St #23...
6,PAST SALE,2024-09-30,Condo/Co-op,18201 Collins Ave #4308,Sunny Isles Beach,FL,33160.0,1550000.0,2.0,2.5,TRUMP ROYALE CONDO,1570.0,NaN,2008.0,NaN,987.0,1587.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11559748,N,Y,25.944933,-80.120001,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Mariana Shulga,"Mariana Shulga, LLC.",None,None,Gustavo Strallnikoff,Dezer Platinum Realty LLC,None,None,7,blue,TRUMP ROYALE CONDO at 18201 Collins Ave #4308 ...
7,PAST SALE,2024-09-30,Condo/Co-op,3400 SW 27th Ave #605,Coconut Grove,FL,33133.0,1450000.0,2.0,2.5,THE TOWER RESIDENCES,1540.0,NaN,2001.0,NaN,942.0,2729.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/3400-SW-27th-A...,MARMLS,A11506988,N,Y,25.729664,-80.237811,https://

In [42]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: MURANO AT PORTOFINO CONDO closing tops Miami-Dade County weekly condo sales 
DEK: Top 10 sales ranged from $1,290,000M to $6,850,000M
FEATURED HED:
SEO HED: Miami-Dade County County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Miami-Dade County condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, weekly condo sales, Miami Beach, Aventura, Coconut Grove, Miami, Sunny Isles Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County County’s NEWS PEG HERE.

Brokers closed 120 condo sales totaling $75,034,917 million from Sep. 29th to Oct. 6th. The previous week, brokers closed 120 condo sales totaling $93,800,000.

Last week’s units sold for an average of $625,291, lower than the $656,812 ave

In [43]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [44]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [45]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [46]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-10-04,Condo/Co-op,1000 S Pointe Dr #301,Miami Beach,FL,33139.0,6850000.0,3.0,4.5,MURANO AT PORTOFINO CONDO,3365.0,NaN,2002.0,NaN,2036.0,4631.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,MARMLS,A11569150,N,Y,25.768527,-80.137591,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,Denise Santos,"Compass Florida, LLC.",None,None,Sildy Cervera,Cervera Real Estate Inc.,None,None,1,orange,MURANO AT PORTOFINO CONDO at 1000 S Pointe Dr ...
1,PAST SALE,2024-09-30,Condo/Co-op,1425 Brickell Ave Unit 57E,Miami,FL,33131.0,4500000.0,4.0,4.0,MILLENNIUM TOWER RESIDENCE,3357.0,NaN,2003.0,NaN,1340.0,4673.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1425-Brickell-...,MARMLS,A11578904,N,Y,25.759132,-80.191829,https://www.redfin.com/FL/Miami/1425-Brickell-...,Darren Weiner,Celebrity Advisors,None,None,Shakira Sanchez,Douglas Elliman,None,None,2,blue,MILLENNIUM TOWER RESIDENCE at 1425 Brickell Av...
2,PAST SALE,2024-10-03,Condo/Co-op,2127 Brickell Ave #3405,Miami,FL,33129.0,2475000.0,3.0,3.0,BRISTOL TOWER CONDO,2310.0,NaN,1993.0,NaN,1071.0,3902.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2127-Brickell-...,MARMLS,A11613620,N,Y,25.752160,-80.199342,https://www.redfin.com/FL/Miami/2127-Brickell-...,Liz Hogan,"Compass Florida, LLC.",None,None,Mariana Niro,Serhant,None,None,3,blue,BRISTOL TOWER CONDO at 2127 Brickell Ave #3405...
3,PAST SALE,2024-10-04,Condo/Co-op,300 Biscayne Blvd Way #702,Miami,FL,33131.0,2400000.0,2.0,3.0,Aston Martin,1474.0,NaN,2024.0,NaN,1628.0,2588.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/300-Biscayne-B...,MARMLS,A11594800,N,Y,25.770777,-80.187521,https://www.redfin.com/FL/Miami/300-Biscayne-B...,Mickael Lancri,Mickael Lancri PA,None,None,NON-MLS MEMBER,MAR NON MLS MEMBER,None,None,4,blue,Aston Martin at 300 Biscayne Blvd Way #702 in ...
4,PAST SALE,2024-10-01,Condo/Co-op,801 S Miami Ave Ph 5601,Miami,FL,33130.0,2100000.0,3.0,3.5,801 SMA RESIDENCES CONDO,1996.0,NaN,2018.0,NaN,1052.0,3250.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/801-S-Miami-Av...,MARMLS,A11595899,N,Y,25.765990,-80.193097,https://www.redfin.com/FL/Miami/801-S-Miami-Av...,Reid Heidenry,One Sotheby's International Re,Marko Gojanovic,One Sotheby's International Re,Marko Gojanovic,One Sotheby's International Re,None,None,5,blue,801 SMA RESIDENCES CONDO at 801 S Miami Ave Ph...
5,PAST SALE,2024-10-01,Condo/Co-op,3301 NE 183rd St #2306,Aventura,FL,33160.0,1745000.0,4.0,4.5,THE PENINSULA II CONDO,3524.0,NaN,2007.0,NaN,495.0,3055.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/3301-NE-183...,MARMLS,A11560683,N,Y,25.946197,-80.137058,https://www.redfin.com/FL/Aventura/3301-NE-183...,Lai Ming Lam,United Realty Group Inc,None,None,Geraldine Dayan,"Compass Florida, LLC.",None,None,6,blue,THE PENINSULA II CONDO at 3301 NE 183rd St #23...
6,PAST SALE,2024-09-30,Condo/Co-op,18201 Collins Ave #4308,Sunny Isles Beach,FL,33160.0,1550000.0,2.0,2.5,TRUMP ROYALE CONDO,1570.0,NaN,2008.0,NaN,987.0,1587.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11559748,N,Y,25.944933,-80.120001,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Mariana Shulga,"Mariana Shulga, LLC.",None,None,Gustavo Strallnikoff,Dezer Platinum Realty LLC,None,None,7,blue,TRUMP ROYALE CONDO at 18201 Collins Ave #4308 ...
7,PAST SALE,2024-09-30,Condo/Co-op,3400 SW 27th Ave #605,Coconut Grove,FL,33133.0,1450000.0,2.0,2.5,THE TOWER RESIDENCES,1540.0,NaN,2001.0,NaN,942.0,2729.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/3400-SW-27th-A...,MARMLS,A11506988,N,Y,25.729664,-80.237811,https://

In [47]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}")

print(story_checklist)


HED: MURANO AT PORTOFINO CONDO closing tops Miami-Dade County weekly condo sales 
DEK: Top 10 sales ranged from $1,290,000M to $6,850,000M
FEATURED HED:
SEO HED: Miami-Dade County County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Miami-Dade County condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, weekly condo sales, Miami Beach, Aventura, Coconut Grove, Miami, Sunny Isles Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County County’s NEWS PEG HERE.

Brokers closed 120 condo sales totaling $75,034,917 million from Sep. 29th to Oct. 6th. The previous week, brokers closed 120 condo sales totaling $93,800,000.

Last week’s units sold for an average of $625,291, lower than the $656,812 ave

In [55]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/Miami-Beach/1000-S-Pointe-Dr-33139/unit-301/home/43443778


In [56]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Miami/1425-Brickell-Ave-33131/unit-57E/home/43295579


In [17]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Miami-Beach/2301-Collins-Ave-33139/unit-537/home/42784160


In [58]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Miami/700-NE-26th-Ter-33137/unit-4606/home/191583478


In [59]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Miami-Beach/1000-S-Pointe-Dr-33139/unit-301/home/43443778


In [60]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Aventura/3301-NE-183rd-St-33160/unit-2306/home/43403058


In [54]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-10-04,Condo/Co-op,1000 S Pointe Dr #301,Miami Beach,FL,33139.0,6850000.0,3.0,4.5,MURANO AT PORTOFINO CONDO,3365.0,NaN,2002.0,NaN,2036.0,4631.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,MARMLS,A11569150,N,Y,25.768527,-80.137591,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,Denise Santos,"Compass Florida, LLC.",None,None,Sildy Cervera,Cervera Real Estate Inc.,None,None,1,orange,MURANO AT PORTOFINO CONDO at 1000 S Pointe Dr ...
1,PAST SALE,2024-09-30,Condo/Co-op,1425 Brickell Ave Unit 57E,Miami,FL,33131.0,4500000.0,4.0,4.0,MILLENNIUM TOWER RESIDENCE,3357.0,NaN,2003.0,NaN,1340.0,4673.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1425-Brickell-...,MARMLS,A11578904,N,Y,25.759132,-80.191829,https://www.redfin.com/FL/Miami/1425-Brickell-...,Darren Weiner,Celebrity Advisors,None,None,Shakira Sanchez,Douglas Elliman,None,None,2,blue,MILLENNIUM TOWER RESIDENCE at 1425 Brickell Av...
2,PAST SALE,2024-10-03,Condo/Co-op,2127 Brickell Ave #3405,Miami,FL,33129.0,2475000.0,3.0,3.0,BRISTOL TOWER CONDO,2310.0,NaN,1993.0,NaN,1071.0,3902.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2127-Brickell-...,MARMLS,A11613620,N,Y,25.752160,-80.199342,https://www.redfin.com/FL/Miami/2127-Brickell-...,Liz Hogan,"Compass Florida, LLC.",None,None,Mariana Niro,Serhant,None,None,3,blue,BRISTOL TOWER CONDO at 2127 Brickell Ave #3405...
3,PAST SALE,2024-10-04,Condo/Co-op,300 Biscayne Blvd Way #702,Miami,FL,33131.0,2400000.0,2.0,3.0,Aston Martin,1474.0,NaN,2024.0,NaN,1628.0,2588.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/300-Biscayne-B...,MARMLS,A11594800,N,Y,25.770777,-80.187521,https://www.redfin.com/FL/Miami/300-Biscayne-B...,Mickael Lancri,Mickael Lancri PA,None,None,NON-MLS MEMBER,MAR NON MLS MEMBER,None,None,4,blue,Aston Martin at 300 Biscayne Blvd Way #702 in ...
4,PAST SALE,2024-10-01,Condo/Co-op,801 S Miami Ave Ph 5601,Miami,FL,33130.0,2100000.0,3.0,3.5,801 SMA RESIDENCES CONDO,1996.0,NaN,2018.0,NaN,1052.0,3250.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/801-S-Miami-Av...,MARMLS,A11595899,N,Y,25.765990,-80.193097,https://www.redfin.com/FL/Miami/801-S-Miami-Av...,Reid Heidenry,One Sotheby's International Re,Marko Gojanovic,One Sotheby's International Re,Marko Gojanovic,One Sotheby's International Re,None,None,5,blue,801 SMA RESIDENCES CONDO at 801 S Miami Ave Ph...
5,PAST SALE,2024-10-01,Condo/Co-op,3301 NE 183rd St #2306,Aventura,FL,33160.0,1745000.0,4.0,4.5,THE PENINSULA II CONDO,3524.0,NaN,2007.0,NaN,495.0,3055.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/3301-NE-183...,MARMLS,A11560683,N,Y,25.946197,-80.137058,https://www.redfin.com/FL/Aventura/3301-NE-183...,Lai Ming Lam,United Realty Group Inc,None,None,Geraldine Dayan,"Compass Florida, LLC.",None,None,6,blue,THE PENINSULA II CONDO at 3301 NE 183rd St #23...
6,PAST SALE,2024-09-30,Condo/Co-op,18201 Collins Ave #4308,Sunny Isles Beach,FL,33160.0,1550000.0,2.0,2.5,TRUMP ROYALE CONDO,1570.0,NaN,2008.0,NaN,987.0,1587.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11559748,N,Y,25.944933,-80.120001,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Mariana Shulga,"Mariana Shulga, LLC.",None,None,Gustavo Strallnikoff,Dezer Platinum Realty LLC,None,None,7,blue,TRUMP ROYALE CONDO at 18201 Collins Ave #4308 ...
7,PAST SALE,2024-09-30,Condo/Co-op,3400 SW 27th Ave #605,Coconut Grove,FL,33133.0,1450000.0,2.0,2.5,THE TOWER RESIDENCES,1540.0,NaN,2001.0,NaN,942.0,2729.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/3400-SW-27th-A...,MARMLS,A11506988,N,Y,25.729664,-80.237811,https://

## Time on Market Calculator

In [62]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 5, 10) ## List (Earlier) date
date2 = datetime(2024, 9, 30) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

143


## Clean CSV for Datawrapper Chart

In [63]:
chart_df = df_top_ten

In [64]:
chart_df = chart_df.fillna(" ")

In [65]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [66]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [67]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [68]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [69]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [70]:
csv_date_string = today.strftime("%m_%d_%Y")

In [71]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")